# Notebook with preprocessing collected data

In [1]:
import numpy as np
import re
import pandas as pd

Данные были собраны с фильтрацией по количеству комнат, так что, объединяя собранные csv файлы, будем добавлять дополнительный столбец с количеством комнат.
Всего было собрано 6 датасетов (от 1-комн. квартир до 6-комн. квартир)

The data was collected with filtering by the number of rooms, so during concatenating the collected csv files, we will add an additional column with the number of rooms.
In total, 6 datasets were collected (from 1 room apartments up to 6 rooms apartments).

In [2]:
DATA_PATH = "./data/"

In [3]:
df_cian = pd.read_csv(DATA_PATH + "cian_houses_1_flats.csv")
df_cian["room_count"] = 1

for i in range(2, 6 + 1):
    df_tmp = pd.read_csv(DATA_PATH + f"cian_houses_{i}_flats.csv")
    df_tmp["room_count"] = i
    df_cian = pd.concat([df_cian, df_tmp], ignore_index=True, sort=False)

df_cian.head()

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
0,18374400,"Москва, СВАО, р-н Марьина роща, Тэйт жилой ком...",55.80253,37.620945,Марьина Роща,34,49,38.28,11.8,unknown,...,3.12,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/302789697/,1
1,49954800,"Москва, ЦАО, р-н Таганский, Славянская пл., 2/5с1",55.752541,37.635727,Китай,7,8,53.20,unknown,24.1,...,3.0,unknown,Монолитно-кирпичный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/301496936/,1
2,8170000,"Москва, НАО (Новомосковский), Филимонковское п...",55.552637,37.337172,Филатов Луг,2,5,31.00,unknown,10.0,...,3.0,Во двор,unknown,Нет информации,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/303552909/,1
3,20206500,"Москва, СВАО, р-н Останкинский, ул. Годовикова...",55.810466,37.624247,Алексеевская,34,37,28.50,21.4,unknown,...,3.01,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,КОРТРОС,https://www.cian.ru/sale/flat/306568424/,1
4,17119620,"Москва, ЮАО, р-н Даниловский, Автозаводская ул...",55.706597,37.632285,Тульская,6,20,38.82,10.6,18.6,...,2.85,Во двор,Монолитный,unknown,unknown,unknown,Новостройка,unknown,https://www.cian.ru/sale/flat/305679800/,1


In [4]:
df_cian.shape

(8053, 22)

Удалим дубликаты, если они есть

In [5]:
df_cian.drop_duplicates()

df_cian.shape

(8053, 22)

Разделим на численные и категориальные фитчи

In [6]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != 'object']
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == 'object']

In [7]:
numerical_features

['floor', 'floor_count', 'square', 'room_count']

В `numerical_features` должны присутствовать еще поля `price`, `geo_lat`, `geo_lng`, `living square`, `kitchen_square`, `ceiling_height`.


Если с последними 3-я полями понятно, что могут присутствовать `unknown` значения, то с первыми 3-я надо разобраться.

Рассмотрим `price`

In [8]:
type(df_cian["price"][0]), type(df_cian["geo_lat"][0]), type(df_cian["geo_lng"][0]), 

(int, float, float)

тип данных &mdash; str, посмотрим м.б. присутствуют неизвестные значения

In [9]:
df_cian[df_cian["price"] == "unknown"]

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
2733,unknown,"Москва, НАО (Новомосковский), № 82 квартал, Но...",unknown,unknown,Филатов Луг,7,7,58.4,unknown,11.4,...,2.7,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,Класс,https://www.cian.ru/sale/flat/296320615/,2
2734,unknown,"Москва, НАО (Новомосковский), № 82 квартал, Но...",unknown,unknown,Филатов Луг,7,7,58.4,unknown,11.4,...,2.7,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,Класс,https://www.cian.ru/sale/flat/299551896/,2


Видно, что произошла ошибка при загрузке данных.

Можно и удалить эти данные, но так как присутствует множество остальных значений, попробуем восстановить пропуски в ручную.

In [10]:
# If you want to delete data with "unknown" price
# df_cian = df_cian.drop(df_cian[df_cian["price"] == "unknown"].index)

In [11]:
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/296320615/", "price"] = "16544720"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/296320615/", "geo_lat"] = "55.571953"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/296320615/", "geo_lng"] = "37.383118"

df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "price"] = "16650000"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "location"] = "Москва, НАО (Новомосковский), Внуковское поселение, ул. Самуила Маршака, 23"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "geo_lat"] = "55.634116"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "geo_lng"] = "37.320039"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "metro"] = "Рассказовка"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "floor"] = 17
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "floor_count"] = 17
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "square"] = 59.0
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "living_square"] = 32.0
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "kitchen_square"] = 13.
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "ceiling_height"] = 2.8
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "view"] = "На улицу и двор"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "heating"] = "Центральное"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "breakdown"] = "Нет"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "house_type"] = "Панельный"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "year"] = 2016
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "author"] = "unknown"
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/", "accomodation_type"] = "Вторичка"

In [12]:
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/296320615/"]


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
2733,16544720,"Москва, НАО (Новомосковский), № 82 квартал, Но...",55.571953,37.383118,Филатов Луг,7,7,58.4,unknown,11.4,...,2.7,На улицу,Монолитно-кирпичный,unknown,unknown,unknown,Новостройка,Класс,https://www.cian.ru/sale/flat/296320615/,2


In [13]:
df_cian.loc[df_cian["url"] == "https://www.cian.ru/sale/flat/299551896/"]

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
2734,16650000,"Москва, НАО (Новомосковский), Внуковское посел...",55.634116,37.320039,Рассказовка,17,17,59.0,32.0,13.0,...,2.8,На улицу и двор,Панельный,Центральное,Нет,unknown,Вторичка,unknown,https://www.cian.ru/sale/flat/299551896/,2


Исправим типы данных

In [14]:
df_cian["price"] = df_cian["price"].apply(lambda x: int(x))
df_cian["geo_lat"] = df_cian["geo_lat"].apply(lambda x: float(x))
df_cian["geo_lng"] = df_cian["geo_lng"].apply(lambda x: float(x))

Посмотрим на численные и категориальные фитчи снова

In [15]:
numerical_features = [f for f in df_cian.columns if df_cian.dtypes[f] != 'object']
categorical_features = [f for f in df_cian.columns if df_cian.dtypes[f] == 'object']

In [16]:
numerical_features

['price', 'geo_lat', 'geo_lng', 'floor', 'floor_count', 'square', 'room_count']

Численные фитчи выправились, в категориальных остались только те, в которых может присутствовать "unknown"

In [17]:
df_cian["metro"].unique()

array(['Марьина Роща', 'Китай', 'Филатов Луг', 'Алексеевская', 'Тульская',
       'Трикотажная', 'ЦСКА', 'Пыхтино', 'Толстопальцево',
       'Улица 1905 года', 'Автозаводская', 'Раменки', 'Бибирево',
       'Маяковская', 'Нижегородская', 'Белорусская', 'Шелепиха',
       'Фонвизинская', 'Новокузнецкая', 'Народное Ополчение',
       'Улица Скобелевская', 'Нахимовский проспект', 'unknown',
       'Сухаревская', 'Бауманская', 'ВДНХ', 'Серпуховская', 'Опалиха',
       'Деловой центр', 'Калужская', 'Павшино', 'Пролетарская',
       'Кунцевская', 'Мичуринский проспект', 'Беляево', 'Фили',
       'Молжаниново', 'Молодёжная', 'Давыдково', 'Медведково',
       'Железнодорожная', 'Воробьёвы горы', 'Павелецкая', 'Хорошёво',
       'Щёлковская', 'Москва', 'Новокосино', 'Третьяковская',
       'Дмитровская', 'Прокшино', 'Перово', 'Измайловская', 'Борисово',
       'Волгоградский проспект', 'Минская', 'Аминьевская', 'Крылатское',
       'Зеленоград', 'Варшавская', 'Воронцовская', 'Котельники',
     

Присутствуют значения содержащие количество минут до станции метро. Выберем и посмотрим на них.

In [18]:
tmp = df_cian["metro"].str.contains(r"\s\d+\sмин", regex=True).to_frame()
df_cian[tmp["metro"] == True]["metro"]

659     Терехово 16 мин
685     Терехово 16 мин
696     Терехово 18 мин
1990    Терехово 17 мин
2044    Терехово 19 мин
2358    Терехово 23 мин
2363    Терехово 23 мин
2384    Терехово 23 мин
2582    Терехово 23 мин
2615    Терехово 23 мин
2926    Терехово 18 мин
3138    Терехово 21 мин
3424    Терехово 23 мин
3436    Терехово 23 мин
3496     Терехово 5 мин
3508    Терехово 23 мин
3553     Терехово 8 мин
3597    Терехово 18 мин
4029    Терехово 17 мин
4710     Терехово 7 мин
4861    Терехово 23 мин
4951    Терехово 22 мин
5666    Терехово 23 мин
6156    Терехово 18 мин
6434     Терехово 5 мин
Name: metro, dtype: object

Такое произошло только со станцией Терехово. Исправим данную ситуацию, оставив только название станции метро.

In [19]:
tmp = df_cian["metro"].str.contains(r"\s\d+\sмин", regex=True).to_frame()
df_cian.loc[tmp["metro"] == True, "metro"] = df_cian[tmp["metro"] == True]["metro"].apply(lambda x: x.split()[0])
df_cian[tmp["metro"] == True]["metro"]

659     Терехово
685     Терехово
696     Терехово
1990    Терехово
2044    Терехово
2358    Терехово
2363    Терехово
2384    Терехово
2582    Терехово
2615    Терехово
2926    Терехово
3138    Терехово
3424    Терехово
3436    Терехово
3496    Терехово
3508    Терехово
3553    Терехово
3597    Терехово
4029    Терехово
4710    Терехово
4861    Терехово
4951    Терехово
5666    Терехово
6156    Терехово
6434    Терехово
Name: metro, dtype: object

Исправлено.

In [20]:
df_cian[(df_cian["square"] <= 10.0) & (df_cian["room_count"] == 1)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
719,2990000,"Москва, ВАО, р-н Северное Измайлово, Сиреневый...",55.803155,37.785009,Щёлковская,2,6,9.6,unknown,2.0,...,unknown,unknown,Блочный,unknown,unknown,unknown,Вторичка / Апартаменты,unknown,https://www.cian.ru/sale/flat/301136222/,1


In [21]:
df_cian[(df_cian["square"] <= 20.0) & (df_cian["room_count"] == 2)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count


In [22]:
df_cian[(df_cian["square"] <= 30.0) & (df_cian["room_count"] == 3)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count
3110,1650000,"Московская область, Пушкинский городской округ...",55.971053,37.932351,unknown,3,5,29.0,unknown,10.0,...,2.65,Во двор,Кирпичный,Центральное,Нет,unknown,Вторичка,unknown,https://ivanteyevka.cian.ru/sale/flat/304682314/,3


In [23]:
df_cian[(df_cian["square"] <= 40.0) & (df_cian["room_count"] == 4)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count


In [24]:
df_cian[(df_cian["square"] <= 50.0) & (df_cian["room_count"] == 5)].sort_values(by="square")


,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count


In [25]:
df_cian[(df_cian["square"] <= 60.0) & (df_cian["room_count"] == 6)].sort_values(by="square")

,price,location,geo_lat,geo_lng,metro,floor,floor_count,square,living_square,kitchen_square,...,ceiling_height,view,house_type,heating,breakdown,parking,accomodation_type,author,url,room_count


Save merged data

In [26]:
df_cian.to_csv(DATA_PATH + "cian_houses_dataset.csv", index=False)